In [54]:
# Instrucciones para implementar un sistema RAG con Langchain y ChromaDB

# Paso 1: Instalar las dependencias necesarias
# Ejecuta el siguiente comando en tu terminal o entorno virtual para instalar las bibliotecas requeridas:
# pip install langchain chromadb openai requests pandas textblob

import pandas as pd
from langchain.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain.agents import Tool, initialize_agent
from langchain.prompts import PromptTemplate
from langchain.agents import AgentType
import openai
from textblob import TextBlob
import requests

In [55]:
# Configura tu clave de API de OpenAI
OPENAI_API_KEY = ""  # Reemplaza con tu clave de API válida

In [56]:
# Paso 2: Cargar el archivo CSV y preparar los datos
file_path = "hidrogeno_verde.csv"  # Ajusta la ruta
hidrogeno_df = pd.read_csv(file_path)

In [57]:
# Preparar los documentos para ChromaDB
documents = hidrogeno_df[['text', 'title', 'date', 'url', 'media_outlet']].copy()
documents['content'] = documents['title'] + " " + documents['text']
documents['metadata'] = documents[['date', 'url', 'media_outlet']].to_dict(orient='records')
texts = documents['content'].tolist()
metadatas = documents['metadata'].tolist()

In [58]:
# Paso 3: Inicializar ChromaDB
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
chroma_db = Chroma.from_texts(texts=texts, metadatas=metadatas, embedding=embeddings, persist_directory="./chromadb_hidrogeno")

In [59]:
# Paso 4: Definir herramientas

def search_chromadb(query):
    results = chroma_db.similarity_search(query, k=3)  # Reducir el número de documentos devueltos
    return "\n".join([str(result.metadata) + "\n" + result.page_content[:500] for result in results])

def wikipedia_search(query):
    response = requests.get("https://es.wikipedia.org/w/api.php", params={
        "action": "query",
        "format": "json",
        "titles": query,
        "prop": "extracts",
        "exintro": True
    })
    pages = response.json().get('query', {}).get('pages', {})
    page = next(iter(pages.values()), {})
    return page.get('extract', "No se encontró información en Wikipedia.")

def sentiment_analysis(query):
    blob = TextBlob(query)
    polarity = blob.sentiment.polarity
    if polarity > 0:
        return "El sentimiento es positivo."
    elif polarity < 0:
        return "El sentimiento es negativo."
    else:
        return "El sentimiento es neutral."

def math_tool(query):
    try:
        result = eval(query)
        return f"El resultado del cálculo es: {result}"
    except Exception as e:
        return f"No pude resolver el cálculo: {e}"

def filter_by_dates(start_date, end_date):
    filtered = hidrogeno_df[(hidrogeno_df['date'] >= start_date) & (hidrogeno_df['date'] <= end_date)]
    return filtered[['title', 'text', 'date']].to_dict(orient='records')

def count_people_mentions(news_list):
    from collections import Counter
    import re
    mentions = []
    for news in news_list:
        mentions.extend(re.findall(r'\b[A-Z][a-z]+\s[A-Z][a-z]+\b', news['text']))
    return Counter(mentions).most_common()

def filter_news_by_person(news_list, person_name):
    filtered_news = [news for news in news_list if person_name in news['text']]
    return filtered_news

def extract_personal_data_from_news(news_list, person_name):
    relevant_texts = [news['text'][:500] for news in news_list if person_name in news['text']]  # Limitar longitud del texto devuelto
    return "\n".join(relevant_texts)


In [60]:
# Crear herramientas

tools = [
    Tool(
        name="ChromaDB Search",
        func=search_chromadb,
        description="Busca en la base de datos de ChromaDB los documentos relevantes."
    ),
    Tool(
        name="Wikipedia Search",
        func=wikipedia_search,
        description="Consulta Wikipedia para obtener más información sobre el tema."
    ),
    Tool(
        name="Sentiment Analysis",
        func=sentiment_analysis,
        description="Analiza el sentimiento de un texto."
    ),
    Tool(
        name="Math Tool",
        func=math_tool,
        description="Resuelve cálculos matemáticos simples, por ejemplo, 2+2."
    ),
    Tool(
        name="Filter by Dates",
        func=lambda query: filter_by_dates(query['start_date'], query['end_date']),
        description="Filtra noticias por un rango de fechas."
    ),
    Tool(
        name="Count People Mentions",
        func=lambda query: count_people_mentions(query['news_list']),
        description="Cuenta las menciones de personas en una lista de noticias."
    ),
    Tool(
        name="Filter News by Person",
        func=lambda query: filter_news_by_person(query['news_list'], query['person_name']),
        description="Filtra las noticias que mencionan a una persona específica."
    ),
    Tool(
        name="Extract Personal Data from News",
        func=lambda query: extract_personal_data_from_news(query['news_list'], query['person_name']),
        description="Extrae datos personales de una persona a partir de noticias."
    )
]

In [61]:
# Paso 5: Configurar el agente Langchain
prompt_template = """
Eres un agente avanzado que puede interactuar con herramientas para obtener información y realizar tareas. 
Tienes acceso a las siguientes herramientas:

1. **ChromaDB Search**: Busca en una base de datos de documentos para obtener información relevante.
2. **Wikipedia Search**: Realiza una consulta a Wikipedia para obtener un resumen de un tema.
3. **Sentiment Analysis**: Analiza el sentimiento de un texto (positivo, negativo, neutral).
4. **Math Tool**: Resuelve cálculos matemáticos simples.
5. **Filter by Dates**: Filtra noticias por un rango de fechas.
6. **Count People Mentions**: Cuenta las menciones de personas en una lista de noticias.
7. **Filter News by Person**: Filtra las noticias que mencionan a una persona específica.
8. **Extract Personal Data from News**: Extrae datos personales de una persona a partir de noticias.

Cuando recibas una pregunta, debes decidir qué herramienta usar y cómo responder de manera concisa y precisa. 
Asegúrate de considerar el contexto y usar las herramientas de manera inteligente.
"""
# Crear el PromptTemplate
prompt = PromptTemplate(input_variables=["tools", "query", "agent_scratchpad", "tool_names"], template=prompt_template)

llm = OpenAI(api_key=OPENAI_API_KEY)
# Crear el agente React usando `initialize_agent`
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [63]:
# Paso 6: Probar el agente
query = "Dame el enlace de una foto de Juan Carlos Jobet"


response = agent.invoke(query)
print(response)
    


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


 I should use Wikipedia Search to find more information about Juan Carlos Jobet and then use ChromaDB Search to find the relevant photo.
Action: Wikipedia Search
Action Input: Juan Carlos Jobet
Observation: <p><b>Juan Carlos Jobet Eluchans</b> (Santiago, 14 de octubre de 1975) es un ingeniero comercial y político chileno. Se desempeñó como ministro de Energía desde junio de 2019, y como ministro de Minería desde diciembre de 2020, asumiendo simultáneamente ambos cargos como biministro de Estado de su país hasta marzo de 2022, bajo el segundo gobierno del presidente Sebastián Piñera. Entre julio de 2013 y marzo de 2014 ejerció como ministro del Trabajo y Previsión Social, durante el primer gobierno de Piñera.
</p>
Thought: Now I should use ChromaDB Search to find the relevant photo from the information I found on Wikipedia.
Action: ChromaDB Search
Action Input: Juan Carlos Jobet photo
Observation: {'date': 'Jul 6, 2023 @ 20:00:00.000', 'media_outlet': 'lanacion', 'url': 'https://www.lan